In [2]:
import pandas as pd
from dataset import getdataset
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('../datasets/data.csv')
data

,filepath,filename,class
0,/Users/dunnkers/git/deep-learning/datasets/Ges...,63.jpg,R
1,/Users/dunnkers/git/deep-learning/datasets/Ges...,823.jpg,R
2,/Users/dunnkers/git/deep-learning/datasets/Ges...,1409.jpg,R
3,/Users/dunnkers/git/deep-learning/datasets/Ges...,189.jpg,R
4,/Users/dunnkers/git/deep-learning/datasets/Ges...,77.jpg,R
...,...,...,...
143954,/Users/dunnkers/git/deep-learning/datasets/asl...,W2943.jpg,W
143955,/Users/dunnkers/git/deep-learning/datasets/asl...,W407.jpg,W
143956,/Users/dunnkers/git/deep-learning/datasets/asl...,W1292.jpg,W
143957,/Users/dunnkers/git/deep-learning/datasets/asl...,W361.jpg,W


In [4]:
dataset = getdataset(data)
dataset

Loaded 143959 files from 26 categories.


<BatchDataset shapes: ((None, 224, 224, 3), (None, 26)), types: (tf.float32, tf.float32)>

In [5]:
ims, labels = next(iter(dataset.take(1)))
ims.shape, labels.shape

(TensorShape([32, 224, 224, 3]), TensorShape([32, 26]))

Labels should be one-hot encoded float or integer vectors.

In [56]:
labels[0]

<tf.Tensor: shape=(26,), dtype=float32, numpy=
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [58]:
model = tf.keras.applications.MobileNetV3Small(
    weights=None, classes=26
)

In [59]:
from keras.metrics import top_k_categorical_accuracy
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.optimizers import Adam
def top_5_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=5)

LEARNING_RATE = 1e-4
model.compile(optimizer = Adam(lr=LEARNING_RATE), 
                   loss = 'categorical_crossentropy',
                   metrics = ['categorical_accuracy', top_5_accuracy])

Prediction is at first just a uniform prediction, i.e. all weights equal.

In [60]:
model.predict(x=ims)[0] # first sample of some batch

array([0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154], dtype=float32)

In [61]:
model.test_on_batch(ims, y=labels) # test whether model accepts a batch input

[3.25809645652771, 1.0, 1.0]

In [62]:
model.fit(x=dataset)

  25/4499 [..............................] - ETA: 3:26:34 - loss: 1.4028 - categorical_accuracy: 0.9150 - top_5_accuracy: 0.9463

KeyboardInterrupt: 